<a href="https://colab.research.google.com/github/s-fellner/BSA_Rolex/blob/main/Code/googlecloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test pour l'utilisation d'un modèle AutoML de google cloud :

## Setting environment

In [ ]:
# Installing Pyyaml
!pip install --upgrade pyyaml

# Installing google cloud automl
%pip install google-cloud-automl

# Installing spacy FR
!pip install spacy download fr
!spacy download fr_core_news_sm

     |████████████████████████████████| 645kB 6.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 389kB 7.7MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 


     |████████████████████████████████| 14.7MB 5.7MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp37-none-any.whl size=14727027 sha256=41400c1afab38621144f8fe7b2b56747205b0b596e2465553862b6dc1f13f410
  Stored in directory: /tmp/pip-ephem-wheel-cache-x2hw6_j2/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
# Setting credentials (for google automl) --> need to upload the json key to venv before running
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/clefadmin.json"

##Google automl Natural language API

In [ ]:
# Supported mime_types: 'text/plain', 'text/html'
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#textsnippet
def predict(content):

  from google.cloud import automl

  # You must first create a dataset, using the `eu` endpoint, before you can
  # call other operations such as: list, get, import, delete, etc.
  client_options = {'api_endpoint': 'eu-automl.googleapis.com:443'}

  # TODO(developer): Uncomment and set the following variables
  project_id = '685330484131'
  # (model AI Crowd) model_id = 'TCN3300918624537018368'
  model_id = 'TCN4629621252099670016'

  prediction_client = automl.PredictionServiceClient(client_options=client_options)

  # Get the full path of the model.
  model_full_id = automl.AutoMlClient.model_path(project_id, "eu", model_id)
  text_snippet = automl.TextSnippet(content=content, mime_type="text/plain")
  payload = automl.ExamplePayload(text_snippet=text_snippet)

  response = prediction_client.predict(name=model_full_id, payload=payload)
  return response

In [ ]:
# Testing the API on a prediction
x = predict("J'aime les chats.")

top_score = x.payload[0].classification.score
top_cat = x.payload[0].display_name

print('Niveau de langue :', top_cat, ' avec un indice de confiance de :', round(top_score * 100), '%.')

Niveau de langue : A1  avec un indice de confiance de : 99 %.


In [ ]:
print(x)

payload {
  annotation_spec_id: "8957672852979449856"
  classification {
    score: 0.9932536482810974
  }
  display_name: "A1"
}
payload {
  annotation_spec_id: "2472489389565935616"
  classification {
    score: 0.006280930247157812
  }
  display_name: "A2"
}
payload {
  annotation_spec_id: "7084175407993323520"
  classification {
    score: 0.00032794862636364996
  }
  display_name: "C1"
}
payload {
  annotation_spec_id: "1319567884959088640"
  classification {
    score: 0.00010564188414718956
  }
  display_name: "B1"
}
payload {
  annotation_spec_id: "166646380352241664"
  classification {
    score: 2.86567064904375e-05
  }
  display_name: "B2"
}
payload {
  annotation_spec_id: "5931253903386476544"
  classification {
    score: 3.230391712349956e-06
  }
  display_name: "C2"
}



## Google automl Table API

In [ ]:
#!/usr/bin/env python

# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""This application demonstrates how to perform basic operations on prediction
with the Google AutoML Tables API.
For more information, the documentation at
https://cloud.google.com/automl-tables/docs.
"""

import argparse


def predictables(
    project_id,
    compute_region,
    model_display_name,
    inputs,
    feature_importance=None,
    print=None,
):
    """Make a prediction."""
    # [START automl_tables_predict]

    from google.cloud import automl_v1beta1 as automl

    client = automl.TablesClient(project=project_id, region=compute_region)

    if feature_importance:
        response = client.predict(
            model_display_name=model_display_name,
            inputs=inputs,
            feature_importance=True,
        )
    else:
        response = client.predict(
            model_display_name=model_display_name, inputs=inputs
        )

    if print:
      print("Prediction results:")
      for result in response.payload:
          print(
              "Predicted class name: {}".format(result.tables.value)
          )
          print("Predicted class score: {}".format(result.tables.score))

          if feature_importance:
              # get features of top importance
              feat_list = [
                  (column.feature_importance, column.column_display_name)
                  for column in result.tables.tables_model_column_info
              ]
              feat_list.sort(reverse=True)
              if len(feat_list) < 10:
                  feat_to_show = len(feat_list)
              else:
                  feat_to_show = 10

              print("Features of top importance:")
              for feat in feat_list[:feat_to_show]:
                  print(feat)
        
    return response

    # [END automl_tables_predict]

## Complementary functions

In [ ]:
##### Custom tokenizer function

# Loading every module needed :
import numpy as np
import pandas as pd
import re 
import string


import spacy
sp = spacy.load('fr_core_news_sm')

# Loading our custom cognate list
cognates_df = pd.read_csv("https://raw.githubusercontent.com/s-fellner/BSA_Rolex/main/Data/Cognates_2.1.csv", sep=',')
cognates = cognates_df['cognates'].to_list()

# Create a list of stopwords
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

# Create a list of punctuation marks
punctuations = string.punctuation

def spacy_tokenizer(sentence):

  # Create token object, which is used to create documents with linguistic annotations.
  mytokens = sp(sentence)

  # Lemmatize each token and convert each token into lowercase
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

  # Remove stop words and punctuation
  mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

  mytokens = [word for word in mytokens if len(word)>1]

  # Return preprocessed list of tokens
  return mytokens

In [ ]:
##### Function to shape a sentence accordingly to data format constraints of google automl

def mef(sentence, col):

#Init
  df = pd.DataFrame()
  sen = []

#Adding sentence in a list
  sen.append(sentence)

#Entering the list in the DF
  df[col] = sen

#Create a column "tokenizer" on which the custom tokenizer is applied
  df['tokenized'] = df[col].apply(spacy_tokenizer)

#Init of 3 lists for the additionnal features
  wordnb = []
  cognate = []
  punctuation = []

#Loop to generate our additionnal features to the DF
  x=0
  while x < df.shape[0]:
    cogs = []
    wordnb.append(len(re.findall(r'\w+', df[col][x])))
    punctuation.append(len(re.findall(r'[.?\-",]+', df[col][x])))
    for token in df['tokenized'][x]:
      if token in cognates:
        cogs.append(token)
    cognate.append(cogs)
    x+=1

  df['wordnb'] = wordnb
  df['cognate'] = cognate
  df['cognatenb'] = df.cognate.map(len)
  df['cognateratio'] = df['cognatenb'] / df['wordnb']
  df['punctuation'] = punctuation
  df['token_nb'] = df.tokenized.map(len)
  df['tokenratio'] = df['token_nb'] / df['wordnb']

  return df

In [ ]:
##### Complete prediction function which do 2 predictions (one on natural language and one on table)
# monitor = True if predictions are used in the Monitor model (specific format)
# print = True to give more details in the prediction process

def full_pred(sentence, monitor=None, print=None):

# formating the sentence
  sentence_df = mef(sentence, 'sentence')

# Natural language prediction
  nl_response = predict(sentence)

# best class predicted retrieved
  nl_top_score = nl_response.payload[0].classification.score
  nl_top_cat = nl_response.payload[0].display_name

# Response init
  response={}

# special format if monitor = True : Full scores are needed
  if monitor:
    for entry in nl_response.payload:
      response['nl_' + entry.display_name]=entry.classification.score

# table prediction
  tl_response = predictables(685330484131, 'us-central1', "Train_3000_Feat", {
      "cognatenb": sentence_df['cognatenb'][0].astype(float),
      "cognateratio": sentence_df['cognateratio'][0].astype(float),
      "punctuation": sentence_df['punctuation'][0].astype(float),
      "sentence": sentence_df['sentence'][0],
      "token_nb": sentence_df['token_nb'][0].astype(float),
      "tokenratio": sentence_df['tokenratio'][0].astype(float),
      "wordnb": sentence_df['wordnb'][0].astype(float),
      "cognate": str(sentence_df['cognate'][0]),
      "tokenized": str(sentence_df['tokenized'][0])})

# saving variable init
  tl_top_score = 0.0
  tl_top_cat = ''

# Finding best class and score
  for entry in tl_response.payload:
    if entry.tables.score > tl_top_score:
      tl_top_score = entry.tables.score
      tl_top_cat = entry.tables.value
  
  response['nl_score']=nl_top_score
  response['nl_cat']=nl_top_cat
  response['tl_score']=tl_top_score
  response['tl_cat']=tl_top_cat

# special format if monitor = True : Full scores are needed
  if monitor:
    for entry in tl_response.payload:
      response['tl_' + entry.tables.value]=entry.tables.score

# if print : printing the local scores
  if print:
    print('Natural language response :', round(nl_top_score*100, 1) , '% level' , nl_top_cat)
    print('AutoML Tables response :', round(tl_top_score*100,1) , '% level' , tl_top_cat)

  return response

In [ ]:
# Testing a full prediction
x = full_pred("Il sait bien cuisiner alors en général, il prépare aussi le repas.")
x

{'nl_cat': 'A2',
 'nl_score': 0.7337931394577026,
 'tl_cat': 'B1',
 'tl_score': 0.4346313774585724}

# Train Data Formating for monitoring model



In [ ]:
# Loading the 1800 isolated entries (are not used in the training of NL and Table google automl models)
Train_data_30 = pd.read_csv("https://raw.githubusercontent.com/s-fellner/BSA_Rolex/main/Data/Train_1800.csv", sep=';')
Train_data_30

,sentence,difficulty
0,Nous sommes en vacances.,A1
1,Je m'appelle Jean.,A1
2,Il habite en Italie avec sa femme et ses enfants.,A1
3,il y a plus de 50 pays francophones !,A1
4,Il est danois.,A1
...,...,...
1795,Le problème va du garçon de café tentant inint...,C2
1796,Les lois de programmation mentionnées à l'anté...,C2
1797,"La conscience malheureuse, c'est donc le mauva...",C2
1798,"Comme Monsieur, un informaticien fondateur de ...",C2


In [ ]:
###### Creating a table which contains all predicted scores in NL and Table models
def MonitorFormat(table, column):

# Init
  df = table

  nl_cat = []
  nl_score = []
  tl_cat = []
  tl_score = []

  nl_A1 = []
  nl_A2 = []
  nl_B1 = []
  nl_B2 = []
  nl_C1 = []
  nl_C2 = []

  tl_A1 = []
  tl_A2 = []
  tl_B1 = []
  tl_B2 = []
  tl_C1 = []
  tl_C2 = []
  
  x=0

# full pred for each line
  for line in df[column]:

    pred = full_pred(line, monitor=True)

    print(pred)

    nl_cat.append(pred['nl_cat'])
    nl_score.append(pred['nl_score'])
    tl_cat.append(pred['tl_cat'])
    tl_score.append(pred['tl_score'])

    nl_A1.append(pred['nl_A1'])
    nl_A2.append(pred['nl_A2'])
    nl_B1.append(pred['nl_B1'])
    nl_B2.append(pred['nl_B2'])
    nl_C1.append(pred['nl_C1'])
    nl_C2.append(pred['nl_C2'])

    tl_A1.append(pred['tl_A1'])
    tl_A2.append(pred['tl_A2'])
    tl_B1.append(pred['tl_B1'])
    tl_B2.append(pred['tl_B2'])
    tl_C1.append(pred['tl_C1'])
    tl_C2.append(pred['tl_C2'])

    x+=1
    print(x)
  
  df['nl_cat'] = nl_cat
  df['nl_score'] = nl_score
  df['tl_cat'] = tl_cat
  df['tl_score'] = tl_score

  df['nl_A1']=nl_A1
  df['nl_A2']=nl_A2
  df['nl_B1']=nl_B1
  df['nl_B2']=nl_B2
  df['nl_C1']=nl_C1
  df['nl_C2']=nl_C2

  df['tl_A1']=tl_A1
  df['tl_A2']=tl_A2
  df['tl_B1']=tl_B1
  df['tl_B2']=tl_B2
  df['tl_C1']=tl_C1
  df['tl_C2']=tl_C2

  return df

In [ ]:
# Creating the training datas for the monitoring model
Train_monitored = MonitorFormat(Train_data_30, 'sentence').drop('sentence', axis=1)
Train_monitored.to_csv('Train_monitored_full.csv', sep=',', encoding='utf-8', index=False)
Train_monitored

,difficulty,nl_cat,nl_score,tl_cat,tl_score,nl_A1,nl_A2,nl_B1,nl_B2,nl_C1,nl_C2,tl_A1,tl_A2,tl_B1,tl_B2,tl_C1,tl_C2
0,A1,A1,0.999884,A1,0.726987,9.998840e-01,8.417378e-05,3.120191e-05,3.319248e-07,3.015028e-08,2.716754e-07,7.269869e-01,1.831808e-01,0.080285,0.007050,0.001200,0.001297
1,A1,A1,0.999979,A1,0.883342,9.999791e-01,2.015932e-05,3.085416e-07,1.335841e-07,1.487347e-07,8.816594e-08,8.833418e-01,7.271503e-02,0.031819,0.006981,0.003858,0.001285
2,A1,A1,0.963630,A2,0.353912,9.636304e-01,1.444281e-02,2.629569e-03,4.494883e-04,1.862333e-02,2.243591e-04,2.110665e-01,3.539120e-01,0.304006,0.085112,0.022121,0.023782
3,A1,A1,0.877632,A2,0.409213,8.776325e-01,1.155670e-01,4.910437e-03,9.907108e-04,9.196123e-05,8.073155e-04,2.717735e-01,4.092133e-01,0.267955,0.034808,0.004114,0.012136
4,A1,A1,0.998023,A1,0.884927,9.980233e-01,2.642771e-05,1.879113e-03,2.274874e-05,4.603250e-05,2.486427e-06,8.849270e-01,9.303323e-02,0.021357,0.000538,0.000086,0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,C2,C2,0.869670,B2,0.414980,4.586982e-05,4.385048e-06,1.668787e-04,1.061992e-01,2.391408e-02,8.696696e-01,1.423658e-05,2.006551e-04,0.001674,0.414980,0.306966,0.276166
1796,C2,B2,0.730999,C1,0.476643,9.601335e-06,1.683433e-02,2.284235e-02,7.309988e-01,2.159475e-01,1.336744e-02,2.304863e-08,4.746479e-07,0.000040,0.123495,0.476643,0.399821
1797,C2,C2,0.992014,B2,0.371007,1.579040e-06,2.615370e-07,2.013700e-03,4.644375e-03,1.326214e-03,9.920140e-01,2.026333e-03,1.420265e-02,0.093780,0.371007,0.236924,0.282061
1798,C2,C2,0.968531,B2,0.364583,9.469222e-08,9.932106e-06,3.623310e-04,2.808189e-02,3.014633e-03,9.685311e-01,1.247909e-05,6.414027e-04,0.016718,0.364583,0.280408,0.337638


In [ ]:
##### Monitor model API (google automl table)
def Monitor_Classification(entry):
    response = predictables(685330484131, 'us-central1', "Train_Monitor_full", 
                            {'nl_A1': entry['nl_A1'],
                            'nl_A2': entry['nl_A2'],
                            'nl_B1': entry['nl_B1'],
                            'nl_B2': entry['nl_B2'],
                            'nl_C1': entry['nl_C1'],
                            'nl_C2': entry['nl_C2'],
                            'nl_cat': entry['nl_cat'],
                            'nl_score': entry['nl_score'],
                            'tl_A1': entry['tl_A1'],
                            'tl_A2': entry['tl_A2'],
                            'tl_B1': entry['tl_B1'],
                            'tl_B2': entry['tl_B2'],
                            'tl_C1': entry['tl_C1'],
                            'tl_C2': entry['tl_C2'],
                            'tl_cat': entry['tl_cat'],
                            'tl_score': entry['tl_score']})
    
    to_return = {}

    top_score = 0
    top_cat = ''

    for result in response.payload:
      if result.tables.score > top_score:
        top_score = result.tables.score
        top_cat = result.tables.value

    to_return['top_score'] = top_score
    to_return['top_cat'] = top_cat

    return to_return

In [ ]:
# Testing the monitor classification model
test = full_pred('Ceci est un test.', monitor=True)
print(test)
test_filter = Monitor_Classification(test)
print(test_filter)

{'nl_A1': 0.9956381916999817, 'nl_A2': 0.004288627300411463, 'nl_B2': 5.562098158407025e-05, 'nl_B1': 1.4736462617293e-05, 'nl_C1': 2.393653630861081e-06, 'nl_C2': 4.251049858794431e-07, 'nl_score': 0.9956381916999817, 'nl_cat': 'A1', 'tl_score': 0.7984774708747864, 'tl_cat': 'A1', 'tl_C1': 7.504760287702084e-05, 'tl_A1': 0.7984774708747864, 'tl_B2': 0.0006618588231503963, 'tl_C2': 8.153341332217678e-05, 'tl_B1': 0.040045421570539474, 'tl_A2': 0.16065865755081177}
{'top_score': 0.8608291149139404, 'top_cat': 'A1'}


# Batch Prediction NL / TL

In [ ]:
# Loading the test datas (for AIcrowd submission)
Test_data = pd.read_csv("https://raw.githubusercontent.com/s-fellner/BSA_Rolex/main/Data/Test_data_ORIGINAL.csv", sep=',')
Test_data

,sentence
0,Il est devenu courant de déplorer que la Franc...
1,"Sur les flancs et les derrières de la noce, to..."
2,"J'aime aussi beaucoup sa voix, une voix un peu..."
3,En partenariat avec l'INPES (Institut National...
4,Il se réveilla seulement quand il entendit un ...
...,...
1195,Autrefois vous viviez retirée ; vous fuyiez un...
1196,"On a un prof super cool, genre métissé, cheveu..."
1197,"Il n'engendre aucun mouvement alternatif, ce q..."
1198,Si nous comparons notre époque à la fin de l'e...


In [ ]:
# Init a output DF
to_upload = pd.DataFrame()
results = []

# Loop to predict each line of test DF (full pred only = no monitoring)
x=1
for entry in Test_data['sentence']:
  print(x, entry)
  pred = full_pred(entry)
  if pred['nl_score'] > pred['tl_score']:
    results.append(pred['nl_cat'])
  else:
    results.append(pred['tl_cat'])
  x+=1

# Saving results in a csv
to_upload['difficulty'] = results
to_upload.to_csv('Test_AI.csv', encoding='utf-8', index=False)

1 Il est devenu courant de déplorer que la France ait acquis la place de championne d'Europe des prélèvements obligatoires, expression qui englobe les impôts d'État, les impôts locaux, les taxes et les cotisations sociales : la part du Produit intérieur brut (PIB) que les autorités prélèvent sur la richesse nationale a atteint 48, 4 %, alors que la moyenne d'Europe est à 40, 2 %.
Natural language response : 57.5 % level C1
AutoML Tables response : 47.3 % level C2
2 Sur les flancs et les derrières de la noce, tout le village en rumeur bourdonnait comme un essaim
Natural language response : 30.7 % level B1
AutoML Tables response : 37.8 % level B1
3 J'aime aussi beaucoup sa voix, une voix un peu grave, qu'on n'attend pas chez une fille aussi frêle
Natural language response : 46.5 % level B1
AutoML Tables response : 44.7 % level A2
4 En partenariat avec l'INPES (Institut National de Prévention et d'Éducation pour la Santé), elle a mis en place un programme sur l'addiction aux écrans, qui a

# Batch Prediction with Supervised AI: 

In [ ]:
# Loading the test datas (for AIcrowd submission)
Test_data = pd.read_csv("https://raw.githubusercontent.com/s-fellner/BSA_Rolex/main/Data/Test_data_ORIGINAL.csv", sep=',')
Test_data

,sentence
0,Il est devenu courant de déplorer que la Franc...
1,"Sur les flancs et les derrières de la noce, to..."
2,"J'aime aussi beaucoup sa voix, une voix un peu..."
3,En partenariat avec l'INPES (Institut National...
4,Il se réveilla seulement quand il entendit un ...
...,...
1195,Autrefois vous viviez retirée ; vous fuyiez un...
1196,"On a un prof super cool, genre métissé, cheveu..."
1197,"Il n'engendre aucun mouvement alternatif, ce q..."
1198,Si nous comparons notre époque à la fin de l'e...


In [ ]:
# Init an output DF
to_upload = pd.DataFrame()
results = []

# Full pred followed by monitoring prediction on each line of test df
x=1
for entry in Test_data['sentence']:
  print(x, entry)
  pred = full_pred(entry, True)
  pred_monitored = Monitor_Classification(pred)
  print(pred_monitored['top_cat'])
  results.append(pred_monitored['top_cat'])
  x+=1

# Saving results in a csv
to_upload['difficulty'] = results
to_upload.to_csv('Test_AI.csv', encoding='utf-8', index=False)

1 Il est devenu courant de déplorer que la France ait acquis la place de championne d'Europe des prélèvements obligatoires, expression qui englobe les impôts d'État, les impôts locaux, les taxes et les cotisations sociales : la part du Produit intérieur brut (PIB) que les autorités prélèvent sur la richesse nationale a atteint 48, 4 %, alors que la moyenne d'Europe est à 40, 2 %.
C1
2 Sur les flancs et les derrières de la noce, tout le village en rumeur bourdonnait comme un essaim
C2
3 J'aime aussi beaucoup sa voix, une voix un peu grave, qu'on n'attend pas chez une fille aussi frêle
A2
4 En partenariat avec l'INPES (Institut National de Prévention et d'Éducation pour la Santé), elle a mis en place un programme sur l'addiction aux écrans, qui aborde le fonctionnement du cerveau, à la limite des neurosciences.
C2
5 Il se réveilla seulement quand il entendit un bruit étrange, et quand il ouvrit les yeux, il vit que tous les serpents étaient réveillés.
B1
6 Illusion de l'ignoble qui attir

##Format visualisation

Normal sentence : used as entry for the natural language model

In [ ]:
sentence = 'Ceci est une phrase de niveau indéterminé.'
prediction_NL = predict(sentence)
print('Entry:\n\n',sentence, '\n\n Response : \n\n', prediction_NL)

Entry:

 Ceci est une phrase de niveau indéterminé. 

 Response : 

 payload {
  annotation_spec_id: "2472489389565935616"
  classification {
    score: 0.6792113780975342
  }
  display_name: "A2"
}
payload {
  annotation_spec_id: "5931253903386476544"
  classification {
    score: 0.14852452278137207
  }
  display_name: "C2"
}
payload {
  annotation_spec_id: "8957672852979449856"
  classification {
    score: 0.07145469635725021
  }
  display_name: "A1"
}
payload {
  annotation_spec_id: "7084175407993323520"
  classification {
    score: 0.06139107421040535
  }
  display_name: "C1"
}
payload {
  annotation_spec_id: "1319567884959088640"
  classification {
    score: 0.036636754870414734
  }
  display_name: "B1"
}
payload {
  annotation_spec_id: "166646380352241664"
  classification {
    score: 0.0027816209476441145
  }
  display_name: "B2"
}



Sentence with additionnal features : used as entry for the Tables model

In [ ]:
sentence_df = mef(sentence, 'sentence')
prediction_TL = predictables(685330484131, 'us-central1', "Train_3000_Feat", {
      "cognatenb": sentence_df['cognatenb'][0].astype(float),
      "cognateratio": sentence_df['cognateratio'][0].astype(float),
      "punctuation": sentence_df['punctuation'][0].astype(float),
      "sentence": sentence_df['sentence'][0],
      "token_nb": sentence_df['token_nb'][0].astype(float),
      "tokenratio": sentence_df['tokenratio'][0].astype(float),
      "wordnb": sentence_df['wordnb'][0].astype(float),
      "cognate": str(sentence_df['cognate'][0]),
      "tokenized": str(sentence_df['tokenized'][0])})
print('Entry:')
display(sentence_df)
print('\n Response :\n\n', prediction_NL)

Entry:


,sentence,tokenized,wordnb,cognate,cognatenb,cognateratio,punctuation,token_nb,tokenratio
0,Ceci est une phrase de niveau indéterminé.,"[phrase, niveau, indéterminer]",7,[phrase],1,0.142857,1,3,0.428571



 Response :

 payload {
  annotation_spec_id: "2472489389565935616"
  classification {
    score: 0.6792113780975342
  }
  display_name: "A2"
}
payload {
  annotation_spec_id: "5931253903386476544"
  classification {
    score: 0.14852452278137207
  }
  display_name: "C2"
}
payload {
  annotation_spec_id: "8957672852979449856"
  classification {
    score: 0.07145469635725021
  }
  display_name: "A1"
}
payload {
  annotation_spec_id: "7084175407993323520"
  classification {
    score: 0.06139107421040535
  }
  display_name: "C1"
}
payload {
  annotation_spec_id: "1319567884959088640"
  classification {
    score: 0.036636754870414734
  }
  display_name: "B1"
}
payload {
  annotation_spec_id: "166646380352241664"
  classification {
    score: 0.0027816209476441145
  }
  display_name: "B2"
}



Scores obtained by the Natural Language model and the Tables model : used as entry for the Monitor model

In [ ]:
monitorformat = full_pred(sentence, monitor=True)
monitored = Monitor_Classification(monitorformat)
print('Entry:\n')
display(monitor)
print('\n Response :\n')
display(monitored)

Entry:



{'nl_A1': 0.07145469635725021,
 'nl_A2': 0.6792113780975342,
 'nl_B1': 0.036636754870414734,
 'nl_B2': 0.0027816209476441145,
 'nl_C1': 0.06139107421040535,
 'nl_C2': 0.14852452278137207,
 'nl_cat': 'A2',
 'nl_score': 0.6792113780975342,
 'tl_A1': 0.22998261451721191,
 'tl_A2': 0.19717049598693848,
 'tl_B1': 0.17881768941879272,
 'tl_B2': 0.2389831244945526,
 'tl_C1': 0.0783744528889656,
 'tl_C2': 0.07667158544063568,
 'tl_cat': 'B2',
 'tl_score': 0.2389831244945526}


 Response :



{'top_cat': 'A2', 'top_score': 0.3694843649864197}